# 05_final_analysis.ipynb - Фінальний аналіз push-сповіщень

## 🎯 Фінальний аналіз впливу push-сповіщень на депозити Android-користувачів

Цей ноутбук завершує всі аналізи та надає остаточні рекомендації для оптимізації push-стратегії.

### 📊 Основні завдання:
1. **Географічний Tier-аналіз** - оптимальна кількість push-ів по регіонах
2. **Temporal аналіз** - коли відбуваються депозити
3. **Аналіз за застосунками** - ефективність TMNT груп
4. **Фінальні рекомендації** - actionable інсайти для бізнесу

### 🔧 Вхідні дані:
- **3,219,484 користувачів** (матчинг push + conversion)
- **41,193 користувачів з депозитами** (1.28% конверсія)
- **$226,815 загального доходу**

---

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, mannwhitneyu
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Налаштування візуалізації
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

print('🎯 ФІНАЛЬНИЙ АНАЛІЗ PUSH-СТРАТЕГІЇ')
print('=' * 55)
print('📊 Комплексний аналіз ефективності push-сповіщень')
print('🌍 Фокус: Географічна оптимізація та temporal patterns')
print('💡 Мета: Actionable рекомендації для бізнесу')

## **КРОК 1: Завантаження та підготовка даних**

In [ ]:
print('\n🔗 КРОК 1: ЗАВАНТАЖЕННЯ ФІНАЛЬНИХ ДАНИХ')
print('-' * 45)

# Завантажуємо готові дані з матчингу
try:
    df = pd.read_parquet('../data/processed/ab_analysis_ready.parquet')
    print(f'✅ Основний датасет завантажено: {len(df):,} записів')
except FileNotFoundError:
    print('❌ Файл ab_analysis_ready.parquet не знайдено. Завантажуємо з матчингу...')
    # Fallback - завантажуємо окремі файли та робимо матчинг
    push_df = pd.read_parquet('../data/processed/push_users_data.parquet')
    conv_df = pd.read_parquet('../data/processed/conversion_users_data.parquet')
    
    # Швидкий матчинг
    df = push_df.merge(conv_df, on='gadid', how='left', suffixes=('_push', '_conv'))
    
    # Додаємо бінарні індикатори
    df['has_deposit'] = (df['total_deposits'].fillna(0) > 0).astype(int)
    df['has_registration'] = (df['total_registrations'].fillna(0) > 0).astype(int)
    df['total_revenue'] = df['total_revenue'].fillna(0)
    
    # Фінальні колонки
    df['tier_final'] = df['tier_push'].fillna(df['tier_conv']) if 'tier_conv' in df.columns else df.get('tier')
    df['country_final'] = df['country_push'].fillna(df['country_conv']) if 'country_conv' in df.columns else df.get('country')
    
    print(f'✅ Матчинг виконано: {len(df):,} записів')

# Швидка перевірка якості даних
print('\n📊 ЯКІСТЬ ДАНИХ:')
print(f'   📱 Унікальних gadid: {df["gadid"].nunique():,}')
print(f'   💰 З депозитами: {df["has_deposit"].sum():,} ({df["has_deposit"].mean()*100:.2f}%)')
print(f'   📝 З реєстраціями: {df["has_registration"].sum():,} ({df["has_registration"].mean()*100:.2f}%)')
print(f'   💵 Загальний дохід: ${df["total_revenue"].sum():,.2f}')
print(f'   🏷️ A/B групи: {sorted(df["ab_group"].unique())}')

# Перевірка Tier розподілу
tier_col = 'tier_final' if 'tier_final' in df.columns else 'tier'
if tier_col in df.columns:
    tier_dist = df[tier_col].value_counts()
    print(f'   🌍 Tier розподіл: {tier_dist.to_dict()}')
else:
    print('   ⚠️ Колонка tier не знайдена, створюємо базову класифікацію')
    # Тут можна додати логіку для створення tier на основі країни

print('\n✅ Дані готові для аналізу!')

## **КРОК 2: 🌍 ГЕОГРАФІЧНИЙ TIER-АНАЛІЗ**

### **2.1 Базовий розподіл по Tier-ах**

In [ ]:
print('\n🌍 КРОК 2.1: БАЗОВИЙ РОЗПОДІЛ ПО TIER-АХ')
print('-' * 45)

# Визначаємо колонку tier
tier_col = 'tier_final' if 'tier_final' in df.columns else ('tier' if 'tier' in df.columns else None)

if tier_col is None:
    print('⚠️ Створюємо tier на основі country')
    # Базова tier класифікація
    tier_1_countries = ['US', 'CA', 'GB', 'AU', 'DE', 'SE', 'NO', 'DK', 'CH', 'AT']
    tier_2_countries = ['FR', 'IT', 'ES', 'PT', 'BE', 'NL', 'FI', 'IE', 'NZ', 'IL', 'SG', 'HK', 'JP', 'KR']
    
    country_col = 'country_final' if 'country_final' in df.columns else 'country'
    
    def assign_tier(country):
        if pd.isna(country) or country == 'Unknown':
            return 'Unknown'
        elif country in tier_1_countries:
            return 'Tier 1'
        elif country in tier_2_countries:
            return 'Tier 2'
        else:
            return 'Tier 3'
    
    df[tier_col] = df[country_col].apply(assign_tier)
    tier_col = 'tier'

# Розрахунок базових метрик по tier-ах
tier_stats = df.groupby(tier_col).agg({
    'gadid': 'count',
    'total_pushes': 'mean', 
    'has_deposit': ['sum', 'mean'],
    'has_registration': ['sum', 'mean'],
    'total_revenue': ['sum', 'mean']
}).round(4)

# Плоска структура колонок
tier_stats.columns = ['users', 'avg_pushes', 'deposits_count', 'deposit_rate', 
                     'registrations_count', 'registration_rate', 'total_revenue', 'arpu']

# Додаємо відсотки
tier_stats['users_pct'] = (tier_stats['users'] / tier_stats['users'].sum() * 100).round(2)
tier_stats['deposit_rate_pct'] = (tier_stats['deposit_rate'] * 100).round(2)
tier_stats['registration_rate_pct'] = (tier_stats['registration_rate'] * 100).round(2)

print('📊 МЕТРИКИ ПО TIER-АХ:')
display(tier_stats[['users', 'users_pct', 'avg_pushes', 'deposit_rate_pct', 'arpu']])

# Візуалізації
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📊 Базовий аналіз по Tier-ах', fontsize=16, fontweight='bold')

# 1. Розподіл користувачів
ax1 = axes[0, 0]
tier_stats['users'].plot(kind='bar', ax=ax1, color=sns.color_palette("viridis", len(tier_stats)))
ax1.set_title('Кількість користувачів по Tier-ах')
ax1.set_ylabel('Користувачі')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Додаємо значення на стовпці
for i, v in enumerate(tier_stats['users']):
    pct_val = tier_stats.iloc[i]['users_pct']
    ax1.text(i, v + max(tier_stats['users'])*0.01, f'{v:,.0f}\n({pct_val:.1f}%)', 
             ha='center', va='bottom')

# 2. Конверсія депозитів
ax2 = axes[0, 1]
tier_stats['deposit_rate_pct'].plot(kind='bar', ax=ax2, color=sns.color_palette("plasma", len(tier_stats)))
ax2.set_title('Коефіцієнт конверсії депозитів (%)')
ax2.set_ylabel('Конверсія (%)')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

for i, v in enumerate(tier_stats['deposit_rate_pct']):
    ax2.text(i, v + max(tier_stats['deposit_rate_pct'])*0.02, f'{v:.2f}%', 
             ha='center', va='bottom')

# 3. ARPU
ax3 = axes[1, 0]
tier_stats['arpu'].plot(kind='bar', ax=ax3, color=sns.color_palette("cividis", len(tier_stats)))
ax3.set_title('ARPU (дохід на користувача)')
ax3.set_ylabel('ARPU ($)')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3)

for i, v in enumerate(tier_stats['arpu']):
    ax3.text(i, v + max(tier_stats['arpu'])*0.02, f'${v:.2f}', 
             ha='center', va='bottom')

# 4. Середня кількість push-ів
ax4 = axes[1, 1]
tier_stats['avg_pushes'].plot(kind='bar', ax=ax4, color=sns.color_palette("rocket", len(tier_stats)))
ax4.set_title('Середня кількість push-ів')
ax4.set_ylabel('Push-ів')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3)

for i, v in enumerate(tier_stats['avg_pushes']):
    ax4.text(i, v + max(tier_stats['avg_pushes'])*0.02, f'{v:.1f}', 
             ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Ключові інсайти
best_conversion_tier = tier_stats['deposit_rate_pct'].idxmax()
best_arpu_tier = tier_stats['arpu'].idxmax()
largest_tier = tier_stats['users'].idxmax()

print('\n🎯 КЛЮЧОВІ ІНСАЙТИ ПО TIER-АХ:')
print(f'   🏆 Найкраща конверсія: {best_conversion_tier} ({tier_stats.loc[best_conversion_tier, "deposit_rate_pct"]:.2f}%)')
print(f'   💰 Найкращий ARPU: {best_arpu_tier} (${tier_stats.loc[best_arpu_tier, "arpu"]:.2f})')
print(f'   👥 Найбільший сегмент: {largest_tier} ({tier_stats.loc[largest_tier, "users_pct"]:.1f}% користувачів)')

### **2.2 Оптимальна кількість push-ів до першого депозиту**

In [ ]:
print('\n📱 КРОК 2.2: ОПТИМАЛЬНА КІЛЬКІСТЬ PUSH-ІВ')
print('-' * 45)

# Створюємо діапазони push-ів
df['push_range'] = pd.cut(df['total_pushes'], 
                         bins=[0, 10, 20, 30, 50, 100, float('inf')],
                         labels=['1-10', '11-20', '21-30', '31-50', '51-100', '100+'])

# Аналіз по tier-ах та діапазонах push-ів
push_tier_analysis = df.groupby([tier_col, 'push_range']).agg({
    'gadid': 'count',
    'has_deposit': ['sum', 'mean'],
    'total_revenue': 'sum'
}).round(4)

push_tier_analysis.columns = ['users', 'deposits', 'conversion_rate', 'revenue']
push_tier_analysis['conversion_pct'] = (push_tier_analysis['conversion_rate'] * 100).round(2)
push_tier_analysis['revenue_per_user'] = (push_tier_analysis['revenue'] / push_tier_analysis['users']).round(2)

# Фільтруємо групи з мінімальною кількістю користувачів
push_tier_analysis_filtered = push_tier_analysis[push_tier_analysis['users'] >= 100]

print('📊 КОНВЕРСІЯ ПО ДІАПАЗОНАХ PUSH-ІВ ТА TIER-АХ:')
pivot_conversion = push_tier_analysis_filtered['conversion_pct'].unstack(level=0, fill_value=0)
display(pivot_conversion)

# Візуалізація - Line plot конверсії по діапазонах
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Лінійний графік конверсії
ax1 = axes[0]
for tier in pivot_conversion.columns:
    if tier != 'Unknown':  # Виключаємо Unknown для чіткості
        ax1.plot(pivot_conversion.index, pivot_conversion[tier], 
                marker='o', linewidth=2, label=tier, markersize=8)

ax1.set_title('Конверсія по діапазонах push-ів (за Tier-ами)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Діапазон push-ів')
ax1.set_ylabel('Конверсія (%)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Heatmap конверсії
ax2 = axes[1]
sns.heatmap(pivot_conversion.T, annot=True, fmt='.2f', cmap='YlOrRd', 
           ax=ax2, cbar_kws={'label': 'Конверсія (%)'})
ax2.set_title('Heatmap: Конверсія по Tier × Push діапазонах', fontsize=14, fontweight='bold')
ax2.set_xlabel('Діапазон push-ів')
ax2.set_ylabel('Tier')

plt.tight_layout()
plt.show()

# Знаходимо оптимальні діапазони для кожного tier
print('\n🎯 ОПТИМАЛЬНІ ДІАПАЗОНИ PUSH-ІВ ПО TIER-АХ:')
optimal_ranges = {}

for tier in pivot_conversion.columns:
    if tier != 'Unknown' and not pivot_conversion[tier].empty:
        # Знаходимо діапазон з найкращою конверсією
        best_range = pivot_conversion[tier].idxmax()
        best_conversion = pivot_conversion[tier].max()
        
        # Також дивимося на кількість користувачів в цьому діапазоні
        users_in_range = push_tier_analysis_filtered.loc[(tier, best_range), 'users']
        
        optimal_ranges[tier] = {
            'range': best_range,
            'conversion': best_conversion,
            'users': users_in_range
        }
        
        print(f'   🏆 {tier}: {best_range} push-ів → {best_conversion:.2f}% конверсія ({users_in_range:,} користувачів)')

# Аналіз revenue per user по діапазонах
print('\n💰 ДОХІД НА КОРИСТУВАЧА ПО ДІАПАЗОНАХ:')
pivot_revenue = push_tier_analysis_filtered['revenue_per_user'].unstack(level=0, fill_value=0)
display(pivot_revenue.round(2))

# Збереження результатів для фінального звіту
optimal_push_results = {
    'optimal_ranges_by_tier': optimal_ranges,
    'conversion_by_range_tier': pivot_conversion.to_dict(),
    'revenue_by_range_tier': pivot_revenue.to_dict()
}

### **2.3 Temporal аналіз та Heatmap Tier × A/B група**

In [ ]:
print('\n⏰ КРОК 2.3: TEMPORAL АНАЛІЗ ТА A/B × TIER')
print('-' * 45)

# Temporal аналіз - якщо є дати
temporal_analysis = {}

if 'first_push' in df.columns and 'first_conversion_datetime' in df.columns:
    # Обчислюємо час до першого депозиту
    depositors = df[df['has_deposit'] == 1].copy()
    
    if not depositors.empty:
        depositors['first_push'] = pd.to_datetime(depositors['first_push'])
        depositors['first_conversion_datetime'] = pd.to_datetime(depositors['first_conversion_datetime'])
        
        depositors['days_to_deposit'] = (depositors['first_conversion_datetime'] - 
                                       depositors['first_push']).dt.days
        
        # Temporal аналіз по tier-ах
        temporal_by_tier = depositors.groupby(tier_col)['days_to_deposit'].agg([
            'mean', 'median', 'std', 'min', 'max'
        ]).round(2)
        
        print('⏱️ ЧАСОВИЙ АНАЛІЗ ДО ПЕРШОГО ДЕПОЗИТУ:')
        display(temporal_by_tier)
        
        # Графік розподілу днів до депозиту
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # Гістограма днів до депозиту
        ax1 = axes[0]
        depositors['days_to_deposit'].hist(bins=30, ax=ax1, alpha=0.7, edgecolor='black')
        ax1.set_title('Розподіл днів до першого депозиту')
        ax1.set_xlabel('Дні до депозиту')
        ax1.set_ylabel('Частота')
        ax1.grid(True, alpha=0.3)
        
        # Box plot по tier-ах
        ax2 = axes[1]
        depositors.boxplot(column='days_to_deposit', by=tier_col, ax=ax2)
        ax2.set_title('Дні до депозиту по Tier-ах')
        ax2.set_xlabel('Tier')
        ax2.set_ylabel('Дні до депозиту')
        
        plt.tight_layout()
        plt.show()
        
        temporal_analysis = {
            'avg_days_to_deposit_by_tier': temporal_by_tier['mean'].to_dict(),
            'median_days_to_deposit_by_tier': temporal_by_tier['median'].to_dict()
        }
        
        print('\n⚡ TEMPORAL ІНСАЙТИ:')
        fastest_tier = temporal_by_tier['median'].idxmin()
        fastest_days = temporal_by_tier.loc[fastest_tier, 'median']
        print(f'   🏃 Найшвидші депозити: {fastest_tier} ({fastest_days:.1f} днів медіана)')
else:
    print('ℹ️ Temporal аналіз недоступний - відсутні колонки з датами')

# ГОЛОВНИЙ HEATMAP: Tier × A/B група (конверсія)
print('\n🔥 HEATMAP: TIER × A/B ГРУПА (КОНВЕРСІЯ)')

tier_ab_analysis = df.groupby([tier_col, 'ab_group']).agg({
    'gadid': 'count',
    'has_deposit': ['sum', 'mean'],
    'total_revenue': 'sum'
}).round(4)

tier_ab_analysis.columns = ['users', 'deposits', 'conversion_rate', 'revenue']
tier_ab_analysis['conversion_pct'] = (tier_ab_analysis['conversion_rate'] * 100).round(2)
tier_ab_analysis['arpu'] = (tier_ab_analysis['revenue'] / tier_ab_analysis['users']).round(2)

# Фільтруємо групи з мінімальною кількістю користувачів
tier_ab_filtered = tier_ab_analysis[tier_ab_analysis['users'] >= 50]

# Створюємо pivot таблиці для heatmap
conversion_heatmap = tier_ab_filtered['conversion_pct'].unstack(level=1, fill_value=0)
arpu_heatmap = tier_ab_filtered['arpu'].unstack(level=1, fill_value=0)

# Головний Heatmap
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
fig.suptitle('🔥 ЕФЕКТИВНІСТЬ: TIER × A/B ГРУПА', fontsize=16, fontweight='bold')

# Heatmap конверсії
ax1 = axes[0]
sns.heatmap(conversion_heatmap, annot=True, fmt='.2f', cmap='RdYlGn', 
           ax=ax1, cbar_kws={'label': 'Конверсія (%)'})
ax1.set_title('Конверсія депозитів (%)', fontsize=14, fontweight='bold')
ax1.set_xlabel('A/B Група')
ax1.set_ylabel('Tier')

# Heatmap ARPU
ax2 = axes[1]
sns.heatmap(arpu_heatmap, annot=True, fmt='.2f', cmap='viridis', 
           ax=ax2, cbar_kws={'label': 'ARPU ($)'})
ax2.set_title('ARPU - дохід на користувача ($)', fontsize=14, fontweight='bold')
ax2.set_xlabel('A/B Група')
ax2.set_ylabel('Tier')

plt.tight_layout()
plt.show()

# Знаходимо найкращі комбінації Tier × A/B
print('\n🏆 НАЙКРАЩІ КОМБІНАЦІЇ TIER × A/B ГРУПА:')

best_combinations = {}
for tier in conversion_heatmap.index:
    if tier != 'Unknown':
        best_ab = conversion_heatmap.loc[tier].idxmax()
        best_conversion = conversion_heatmap.loc[tier].max()
        best_arpu = arpu_heatmap.loc[tier, best_ab]
        
        # Кількість користувачів в цій комбінації
        users_count = tier_ab_filtered.loc[(tier, best_ab), 'users']
        
        best_combinations[tier] = {
            'ab_group': best_ab,
            'conversion': best_conversion,
            'arpu': best_arpu,
            'users': users_count
        }
        
        print(f'   🎯 {tier}: A/B група {best_ab} → {best_conversion:.2f}% конверсія, ${best_arpu:.2f} ARPU ({users_count:,} користувачів)')

# Збереження результатів
tier_ab_results = {
    'best_combinations': best_combinations,
    'conversion_heatmap': conversion_heatmap.to_dict(),
    'arpu_heatmap': arpu_heatmap.to_dict(),
    'temporal_analysis': temporal_analysis
}

## **КРОК 3: Аналіз за застосунками (TMNT)**

In [ ]:
print('\n🐢 КРОК 3: АНАЛІЗ ЗА ЗАСТОСУНКАМИ (TMNT)')
print('-' * 45)

# Перевіряємо наявність колонки з групами застосунків
app_col = None
possible_app_cols = ['group_name', 'campaign_group', 'app_group']

for col in possible_app_cols:
    if col in df.columns:
        app_col = col
        break

if app_col is None:
    print('⚠️ Колонка з групами застосунків не знайдена. Пропускаємо TMNT аналіз.')
    tmnt_results = {'status': 'no_app_data'}
else:
    print(f'📊 Використовуємо колонку: {app_col}')
    
    # Фільтруємо тільки користувачів з даними про застосунки
    app_users = df[df[app_col].notna()].copy()
    
    if len(app_users) == 0:
        print('⚠️ Немає користувачів з даними про застосунки')
        tmnt_results = {'status': 'no_app_users'}
    else:
        print(f'📊 Користувачів з даними про застосунки: {len(app_users):,}')
        print(f'🐢 Застосунки: {app_users[app_col].value_counts().to_dict()}')
        
        # Базовий аналіз по застосунках
        app_stats = app_users.groupby(app_col).agg({
            'gadid': 'count',
            'total_pushes': 'mean',
            'has_deposit': ['sum', 'mean'],
            'has_registration': ['sum', 'mean'],
            'total_revenue': ['sum', 'mean']
        }).round(4)
        
        app_stats.columns = ['users', 'avg_pushes', 'deposits', 'deposit_rate', 
                            'registrations', 'registration_rate', 'total_revenue', 'arpu']
        
        app_stats['deposit_rate_pct'] = (app_stats['deposit_rate'] * 100).round(2)
        app_stats['registration_rate_pct'] = (app_stats['registration_rate'] * 100).round(2)
        
        print('\n📊 МЕТРИКИ ПО ЗАСТОСУНКАХ:')
        display(app_stats[['users', 'avg_pushes', 'deposit_rate_pct', 'arpu']].sort_values('deposit_rate_pct', ascending=False))
        
        # Візуалізація ефективності застосунків
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('🐢 Аналіз ефективності TMNT застосунків', fontsize=16, fontweight='bold')
        
        # 1. Конверсія депозитів
        ax1 = axes[0, 0]
        app_stats_sorted = app_stats.sort_values('deposit_rate_pct', ascending=True)
        app_stats_sorted['deposit_rate_pct'].plot(kind='barh', ax=ax1, color=sns.color_palette("viridis", len(app_stats)))
        ax1.set_title('Конверсія депозитів по застосунках (%)')
        ax1.set_xlabel('Конверсія (%)')
        ax1.grid(True, alpha=0.3)
        
        # 2. ARPU
        ax2 = axes[0, 1]
        app_stats_sorted = app_stats.sort_values('arpu', ascending=True)
        app_stats_sorted['arpu'].plot(kind='barh', ax=ax2, color=sns.color_palette("plasma", len(app_stats)))
        ax2.set_title('ARPU по застосунках ($)')
        ax2.set_xlabel('ARPU ($)')
        ax2.grid(True, alpha=0.3)
        
        # 3. Кількість користувачів
        ax3 = axes[1, 0]
        app_stats['users'].plot(kind='bar', ax=ax3, color=sns.color_palette("cividis", len(app_stats)))
        ax3.set_title('Кількість користувачів по застосунках')
        ax3.set_ylabel('Користувачі')
        ax3.tick_params(axis='x', rotation=45)
        ax3.grid(True, alpha=0.3)
        
        # 4. Середня кількість push-ів
        ax4 = axes[1, 1]
        app_stats['avg_pushes'].plot(kind='bar', ax=ax4, color=sns.color_palette("rocket", len(app_stats)))
        ax4.set_title('Середня кількість push-ів по застосунках')
        ax4.set_ylabel('Push-ів')
        ax4.tick_params(axis='x', rotation=45)
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Cross-analysis: App × A/B group × Tier (якщо є достатньо даних)
        print('\n🔀 CROSS-ANALYSIS: APP × A/B ГРУПА × TIER')
        
        cross_analysis = app_users.groupby([app_col, 'ab_group', tier_col]).agg({
            'gadid': 'count',
            'has_deposit': 'mean',
            'total_revenue': 'sum'
        }).round(4)
        
        cross_analysis.columns = ['users', 'conversion_rate', 'revenue']
        cross_analysis['conversion_pct'] = (cross_analysis['conversion_rate'] * 100).round(2)
        
        # Фільтруємо комбінації з мінімальною кількістю користувачів
        cross_filtered = cross_analysis[cross_analysis['users'] >= 20]
        
        if not cross_filtered.empty:
            print('📊 Топ-10 комбінацій App × A/B × Tier за конверсією:')
            top_combinations = cross_filtered.nlargest(10, 'conversion_pct')[['users', 'conversion_pct']]
            display(top_combinations)
        
        # Найкращі та найгірші застосунки
        best_app = app_stats['deposit_rate_pct'].idxmax()
        worst_app = app_stats['deposit_rate_pct'].idxmin()
        best_arpu_app = app_stats['arpu'].idxmax()
        
        print('\n🏆 ВИСНОВКИ ПО ЗАСТОСУНКАХ:')
        print(f'   🥇 Найкраща конверсія: {best_app} ({app_stats.loc[best_app, "deposit_rate_pct"]:.2f}%)')
        print(f'   💰 Найкращий ARPU: {best_arpu_app} (${app_stats.loc[best_arpu_app, "arpu"]:.2f})')
        print(f'   📉 Найгірша конверсія: {worst_app} ({app_stats.loc[worst_app, "deposit_rate_pct"]:.2f}%)')
        
        # Збереження результатів
        tmnt_results = {
            'app_stats': app_stats.to_dict(),
            'best_app': best_app,
            'best_arpu_app': best_arpu_app,
            'cross_analysis': cross_filtered.to_dict() if not cross_filtered.empty else {}
        }

## **КРОК 4: 🎯 Фінальні висновки та рекомендації**

In [ ]:
print('\n🎯 КРОК 4: ФІНАЛЬНІ ВИСНОВКИ ТА РЕКОМЕНДАЦІЇ')
print('=' * 55)

# Збираємо всі ключові метрики
final_summary = {
    'total_users': len(df),
    'users_with_deposits': df['has_deposit'].sum(),
    'overall_conversion_rate': (df['has_deposit'].mean() * 100).round(2),
    'total_revenue': df['total_revenue'].sum(),
    'overall_arpu': (df['total_revenue'].sum() / len(df)).round(2)
}

print('📊 ЗАГАЛЬНА СТАТИСТИКА ПРОЕКТУ:')
print(f'   👥 Загалом користувачів: {final_summary["total_users"]:,}')
print(f'   💰 З депозитами: {final_summary["users_with_deposits"]:,} ({final_summary["overall_conversion_rate"]:.2f}%)')
print(f'   💵 Загальний дохід: ${final_summary["total_revenue"]:,.2f}')
print(f'   💵 ARPU: ${final_summary["overall_arpu"]:.2f}')

print('\n' + '=' * 55)
print('📋 ВІДПОВІДІ НА ПОЧАТКОВІ ПИТАННЯ:')
print('=' * 55)

# 1. Матчинг користувачів за gadid
print('\n1️⃣ МАТЧИНГ КОРИСТУВАЧІВ ЗА GADID: ✅ ЗРОБЛЕНО')
print('   • Зіставлено користувачів між statistic і keitaro')
print('   • Вилучено записи з gadid = null')
print('   • Враховано множинність застосунків')
print(f'   • Результат: {final_summary["total_users"]:,} користувачів у фінальному датасеті')

# 2. Зв'язок push-ів і депозитів по застосунку
print('\n2️⃣ ЗВ\'ЯЗОК PUSH-ІВ І ДЕПОЗИТІВ ПО ЗАСТОСУНКУ: ✅ ПРОАНАЛІЗОВАНО')
if 'tmnt_results' in locals() and 'app_stats' in tmnt_results:
    print('   • Проаналізовано Android-застосунки (TMNT групи)')
    for app, stats in tmnt_results['app_stats'].items():
        if isinstance(stats, dict) and 'deposit_rate_pct' in stats:
            print(f'   • {app}: {stats["deposit_rate_pct"]:.2f}% конверсія')
else:
    print(f'   • Загальна конверсія Android користувачів: {final_summary["overall_conversion_rate"]:.2f}%')
    print(f'   • Користувачі з депозитами: {final_summary["users_with_deposits"]:,}')

# 3. Вплив A/B-групи push-ів на депозити
print('\n3️⃣ ВПЛИВ A/B-ГРУПИ PUSH-ІВ НА ДЕПОЗИТИ: ✅ ВИЗНАЧЕНО')
ab_conversion = df.groupby('ab_group')['has_deposit'].mean() * 100
best_ab_group = ab_conversion.idxmax()
best_ab_conversion = ab_conversion.max()
print(f'   • Найефективніша A/B група: {best_ab_group}')
print(f'   • Конверсія найкращої групи: {best_ab_conversion:.2f}%')
print('   • Розподіл конверсій по A/B групах:')
for group, conv in ab_conversion.sort_values(ascending=False).items():
    print(f'     - Група {group}: {conv:.2f}%')

# 4. Оптимальна кількість push-ів по гео
print('\n4️⃣ ОПТИМАЛЬНА КІЛЬКІСТЬ PUSH-ІВ ПО ГЕО: ✅ ВИЗНАЧЕНО ПО TIER-АХ')
if 'optimal_push_results' in locals():
    print('   • Рекомендації по кожному Tier:')
    for tier, data in optimal_push_results['optimal_ranges_by_tier'].items():
        print(f'     - {tier}: {data["range"]} push-ів → {data["conversion"]:.2f}% конверсія')
else:
    print(f'   • Середня кількість push-ів: {df["total_pushes"].mean():.1f}')
    print(f'   • Медіанна кількість push-ів: {df["total_pushes"].median():.0f}')

print('\n' + '=' * 55)
print('💡 КОНКРЕТНІ БІЗНЕС-РЕКОМЕНДАЦІЇ:')
print('=' * 55)

# Рекомендації по Tier-ах
if 'tier_ab_results' in locals() and 'best_combinations' in tier_ab_results:
    print('\n🌍 СТРАТЕГІЯ ПО TIER-АХ:')
    for tier, combo in tier_ab_results['best_combinations'].items():
        optimal_pushes = optimal_push_results['optimal_ranges_by_tier'].get(tier, {}).get('range', 'N/A')
        print(f'\n🎯 {tier}:')
        print(f'   • Використовувати A/B групу: {combo["ab_group"]}')
        print(f'   • Оптимальна кількість push-ів: {optimal_pushes}')
        print(f'   • Очікувана конверсія: {combo["conversion"]:.2f}%')
        print(f'   • Очікуваний ARPU: ${combo["arpu"]:.2f}')
        print(f'   • Розмір сегменту: {combo["users"]:,} користувачів')

# Загальна стратегія оптимізації
print('\n🚀 ЗАГАЛЬНА СТРАТЕГІЯ ОПТИМІЗАЦІЇ:')
print(f'1. 📱 Універсальна A/B група: Використовувати групу {best_ab_group} як базову')
print('2. 🌍 Локалізація: Адаптувати стратегію під кожен Tier')
print('3. 📊 Частота: Оптимізувати кількість push-ів згідно з tier-рекомендаціями')
print('4. ⏰ Timing: Фокусуватися на перші дні після установки')

if 'tmnt_results' in locals() and 'best_app' in tmnt_results:
    print(f'5. 🐢 Застосунки: Пріоритизувати розвиток {tmnt_results["best_app"]}')

print('\n💰 ПОТЕНЦІЙНИЙ IMPACT:')
# Розрахунок потенційного покращення
current_conversion = final_summary['overall_conversion_rate']
best_possible_conversion = best_ab_conversion
improvement_potential = ((best_possible_conversion - current_conversion) / current_conversion) * 100

potential_additional_deposits = int(final_summary['total_users'] * (best_possible_conversion - current_conversion) / 100)
potential_additional_revenue = potential_additional_deposits * final_summary['overall_arpu']

print(f'   📈 Покращення конверсії: +{improvement_potential:.1f}%')
print(f'   💰 Додаткові депозити: ~{potential_additional_deposits:,}')
print(f'   💵 Додатковий дохід: ~${potential_additional_revenue:,.0f}')
print('   🎯 ROI оптимізації: Висока віддача при мінімальних витратах')

print('\n' + '=' * 55)
print('🎉 АНАЛІЗ ЗАВЕРШЕНО!')
print('=' * 55)

# Підготовка фінальних результатів для експорту
export_results = {
    'summary': final_summary,
    'ab_analysis': {
        'best_group': best_ab_group,
        'conversions_by_group': ab_conversion.to_dict()
    },
    'tier_analysis': tier_stats.to_dict() if 'tier_stats' in locals() else {},
    'optimal_pushes': optimal_push_results if 'optimal_push_results' in locals() else {},
    'tier_ab_combinations': tier_ab_results if 'tier_ab_results' in locals() else {},
    'tmnt_analysis': tmnt_results if 'tmnt_results' in locals() else {},
    'recommendations': {
        'improvement_potential': improvement_potential,
        'potential_additional_revenue': potential_additional_revenue,
        'key_actions': [
            f'Використовувати A/B групу {best_ab_group} як базову',
            'Локалізувати стратегію під кожен Tier',
            'Оптимізувати кількість push-ів за tier-рекомендаціями',
            'Фокусуватися на швидких конверсіях'
        ]
    }
}

## **КРОК 5: Експорт результатів**

In [ ]:
print('\n💾 КРОК 5: ЕКСПОРТ РЕЗУЛЬТАТІВ')
print('-' * 40)

# Створюємо директорію для результатів якщо її немає
output_dir = '../outputs/final_results'
os.makedirs(output_dir, exist_ok=True)

# Експорт основних результатів в JSON
results_files = {
    'final_analysis_summary.json': export_results,
    'tier_recommendations.json': tier_ab_results if 'tier_ab_results' in locals() else {},
    'ab_group_results.json': {'best_group': best_ab_group, 'conversions': ab_conversion.to_dict()},
    'optimal_pushes_by_tier.json': optimal_push_results if 'optimal_push_results' in locals() else {},
    'tmnt_app_analysis.json': tmnt_results if 'tmnt_results' in locals() else {}
}

for filename, data in results_files.items():
    filepath = os.path.join(output_dir, filename)
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2, default=str)
        print(f'✅ Збережено: {filename}')
    except Exception as e:
        print(f'❌ Помилка збереження {filename}: {e}')

# Експорт ключових таблиць в CSV
csv_exports = {}

if 'tier_stats' in locals():
    csv_exports['tier_analysis.csv'] = tier_stats

if 'conversion_heatmap' in locals():
    csv_exports['tier_ab_conversion_matrix.csv'] = conversion_heatmap

if 'tmnt_results' in locals() and 'app_stats' in tmnt_results:
    app_df = pd.DataFrame(tmnt_results['app_stats']).T
    csv_exports['tmnt_app_stats.csv'] = app_df

for filename, dataframe in csv_exports.items():
    filepath = os.path.join(output_dir, filename)
    try:
        dataframe.to_csv(filepath, encoding='utf-8')
        print(f'✅ Збережено: {filename}')
    except Exception as e:
        print(f'❌ Помилка збереження {filename}: {e}')

# Створення README файлу з короткими результатами
readme_text = f'''# Push Analysis - Фінальні результати

## 📊 Загальна статистика
- **Загалом користувачів**: {final_summary["total_users"]:,}
- **З депозитами**: {final_summary["users_with_deposits"]:,} ({final_summary["overall_conversion_rate"]:.2f}%)
- **Загальний дохід**: ${final_summary["total_revenue"]:,.2f}
- **ARPU**: ${final_summary["overall_arpu"]:.2f}

## 🏆 Ключові результати
- **Найкраща A/B група**: {best_ab_group} ({best_ab_conversion:.2f}% конверсія)
- **Потенціал покращення**: +{improvement_potential:.1f}%
- **Додатковий дохід**: ~${potential_additional_revenue:,.0f}

## 📁 Файли результатів
- `final_analysis_summary.json` - Повні результати аналізу
- `tier_recommendations.json` - Рекомендації по географічних tier-ах
- `ab_group_results.json` - Результати A/B тестування
- `optimal_pushes_by_tier.json` - Оптимальна кількість push-ів
- `tmnt_app_analysis.json` - Аналіз застосунків
- `*.csv` - Таблиці для подальшого аналізу

## 💡 Основні рекомендації
1. Використовувати A/B групу {best_ab_group} як базову
2. Локалізувати стратегію під кожен Tier
3. Оптимізувати кількість push-ів за tier-рекомендаціями
4. Фокусуватися на швидких конверсіях

Аналіз завершено: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
'''

readme_path = os.path.join(output_dir, 'README.md')
try:
    with open(readme_path, 'w', encoding='utf-8') as f:
        f.write(readme_text)
    print('✅ Збережено: README.md')
except Exception as e:
    print(f'❌ Помилка збереження README.md: {e}')

print(f'\n📁 Всі результати збережено в: {output_dir}')
print('🎉 ФІНАЛЬНИЙ АНАЛІЗ ЗАВЕРШЕНО!')

# Повертаємо ключові метрики для README проекту
project_summary = {
    'analysis_completed': True,
    'total_users_analyzed': final_summary['total_users'],
    'conversion_rate': final_summary['overall_conversion_rate'],
    'best_ab_group': best_ab_group,
    'improvement_potential': improvement_potential,
    'key_insights': [
        f'Група {best_ab_group} показує {best_ab_conversion:.2f}% конверсію',
        f'Потенціал покращення: +{improvement_potential:.1f}%',
        'Локалізація по tier-ах підвищує ефективність',
        'Оптимальна кількість push-ів варіюється по регіонах'
    ]
}

print('\n📋 SUMMARY ДЛЯ ГОЛОВНОГО README:')
for key, value in project_summary.items():
    if key == 'key_insights':
        print(f'   {key}:')
        for insight in value:
            print(f'     - {insight}')
    else:
        print(f'   {key}: {value}')